In [1]:
import pandas as pd
import datetime
import seaborn as sb
import time
from ipywidgets import interact, fixed
from libs.CovidTimeseriesModel import CovidTimeseriesModel
from libs.CovidDatasets import CovidDatasets
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
Dataset = CovidDatasets()

def model_state(country, state, interventions=None):
   
    ## Constants
    start_time = time.time()
    HOSPITALIZATION_RATE = .0727
    HOSPITALIZED_CASES_REQUIRING_ICU_CARE = .1397
    TOTAL_INFECTED_PERIOD = 12
    MODEL_INTERVAL = 4
    r0 = 2.4
    POP = Dataset.get_population_by_country_state(country, state)
    # Pack all of the assumptions and parameters into a dict that can be passed into the model
    MODEL_PARAMETERS = {
        # Pack the changeable model parameters
        'timeseries': Dataset.get_timeseries_by_country_state(country, state, MODEL_INTERVAL),
        'beds': Dataset.get_beds_by_country_state(country, state),
        'population': POP,
        'projection_iterations': 25, # Number of iterations into the future to project
        'r0': r0,
        'interventions': interventions,
        'hospitalization_rate': HOSPITALIZATION_RATE,
        'case_fatality_rate': .0109341104294479,
        'hospitalized_cases_requiring_icu_care': HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
        # Assumes that anyone who needs ICU care and doesn't get it dies
        'case_fatality_rate_hospitals_overwhelmed': HOSPITALIZATION_RATE * HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
        'hospital_capacity_change_daily_rate': 1.05,
        'max_hospital_capacity_factor': 2.07,
        'initial_hospital_bed_utilization': .6,
        'model_interval': 4, # In days
        'total_infected_period': 12, # In days
        'rolling_intervals_for_current_infected': int(round(TOTAL_INFECTED_PERIOD / MODEL_INTERVAL, 0)),
        'estimated_new_cases_per_death': 32,
        'estimated_new_cases_per_confirmed': 20
    }
    return CovidTimeseriesModel().forecast_region(model_parameters=MODEL_PARAMETERS)

r0 = 2.4

INTERVENTIONS = [
    None, 
    {
        datetime.date(2020, 3, 23): 1.3,
        datetime.date(2020, 4, 20): 1.1,
        datetime.date(2020, 5, 22): 0.8,
        datetime.date(2020, 6, 23): r0
    },
    {
        datetime.date(2020, 3, 23): 1.7,
        datetime.date(2020, 6, 23): r0
    },
    {
        datetime.date(2020, 3, 23): 1.3,
        datetime.date(2020, 3, 31): 0.3,
        datetime.date(2020, 4, 28): 0.2,
        datetime.date(2020, 5,  6): 0.1,
        datetime.date(2020, 5, 10): 0.35,
        datetime.date(2020, 5, 18): r0
    }
]

In [3]:
Dataset.get_timeseries_by_country_state('USA', 'CA', 1)[['date','cases','deaths']]

,date,cases,deaths
0,2020-03-03,25.00,0.00
1,2020-03-04,35.00,1.00
2,2020-03-05,51.00,1.00
3,2020-03-06,59.00,1.00
4,2020-03-07,81.00,1.00
5,2020-03-08,95.00,1.00
6,2020-03-09,101.00,1.00
7,2020-03-10,144.00,2.00
8,2020-03-11,177.00,3.00
9,2020-03-12,221.00,4.00


In [4]:
model_state('USA', 'CA', INTERVENTIONS[0])

,Date,Timestamp,R,Effective R.,Beg. Susceptible,New Inf < C,New Inf < D,New Inf.,Curr. Inf.,Recov. or Died,End Susceptible,Actual Reported,Pred. Hosp.,Cum. Inf.,Cum. Deaths,Avail. Hosp. Beds
0,2020-03-03,1583222400.00,2.40,nan,39511879.12,500,0,500,0.00,0.00,39511879.12,25.00,0.00,nan,nan,28448.80
1,2020-03-07,1583568000.00,3.24,3.24,39510071.01,1620,32,1652,500.00,0.00,39510071.01,81.00,120.10,1651.99,18.06,28448.80
2,2020-03-11,1583910000.00,2.19,2.19,39506365.32,3610,96,3706,2151.99,0.00,39506365.32,177.00,269.40,5357.68,58.58,28448.80
3,2020-03-15,1584255600.00,2.41,2.41,39497255.84,8917,192,9109,5857.68,0.00,39497255.84,426.00,662.26,14467.16,158.19,29871.24
4,2020-03-19,1584601200.00,2.11,2.10,39477537.88,19174,544,19718,14467.16,500.00,39477537.88,897.00,1433.50,34185.12,373.78,31364.80
5,2020-03-23,1584946800.00,2.44,2.44,39428139.23,48055,1344,49399,32533.13,2151.99,39428139.23,2188.00,3591.28,83583.77,913.91,32933.04
6,2020-03-27,1585292400.00,2.40,2.39,39309834.77,118304,0,118304,78226.08,5857.68,39309834.77,nan,8600.73,201888.23,2207.47,34579.69
7,2020-03-31,1585638000.00,2.40,2.39,39027358.40,282476,0,282476,187421.07,14967.16,39027358.40,nan,20536.03,484364.60,5296.10,36308.68
8,2020-04-04,1585983600.00,2.40,2.37,38357734.32,669624,0,669624,450179.48,34685.12,38357734.32,nan,48681.67,1153988.68,19418.67,38124.11
9,2020-04-08,1586329200.00,2.40,2.33,36797593.56,1560141,0,1560141,1070404.91,84083.77,36797593.56,nan,113422.23,2714129.44,52322.51,40030.32
